In [181]:
import pandas as pd
import pickle
from geopy.distance import geodesic

pd.set_option('display.max_columns', None)

# Metro Statistical Area Data (population, GDP, lat/lng)

In [182]:
# Compiled data for population, lat/lng and GDP of a MSA
msa_pickle = '../data/pickled/msa_df.pickle'
with open(msa_pickle, 'rb') as file:
    msa_df = pickle.load(file)

In [183]:
print(len(msa_df))
msa_df.head()

387


,MetroArea,Population,MainCity,lat,lng,GDP_thousands_dollars
0,"Abilene, TX",181591,"Abilene, TX",32.448736,-99.733144,9468978.0
1,"Akron, OH",698398,"Akron, OH",41.081199,-81.518838,44562456.0
2,"Albany, GA",145508,"Albany, GA",31.578507,-84.155741,7312400.0
3,"Albany, OR",131496,"Albany, OR",44.636511,-123.105928,6107649.0
4,"Albany-Schenectady-Troy, NY",904682,"Albany, NY",42.652579,-73.756232,80302855.0


In [184]:
msa_df['Population'].min()

57762

In [185]:
# The MetroArea values are the ones from the population data (2023). 
# Note that the GDP data was from 2022 and I found that there are
# a few new MSAs.
msa_df.loc[msa_df['GDP_thousands_dollars'].isnull()]

,MetroArea,Population,MainCity,lat,lng,GDP_thousands_dollars
11,"Amherst Town-Northampton, MA",162502,"Amherst, MA",42.373222,-72.519854,NaN
45,"Bozeman, MT",126409,"Bozeman, MT",45.679312,-111.037259,NaN
100,"Eagle Pass, TX",57762,"Eagle Pass, TX",28.709143,-100.499521,NaN
150,"Helena, MT",96091,"Helena, MT",46.589145,-112.039106,NaN
180,"Kenosha, WI",167488,"Kenosha, WI",42.584742,-87.821185,NaN
231,"Minot, ND",75742,"Minot, ND",48.232967,-101.292291,NaN
263,"Paducah, KY-IL",102267,"Paducah, KY",37.083389,-88.600048,NaN
271,"Pinehurst-Southern Pines, NC",106898,"Pinehurst, NC",35.195434,-79.469477,NaN
310,"Sandusky, OH",113838,"Sandusky, OH",41.456175,-82.711682,NaN
329,"Slidell-Mandeville-Covington, LA",275583,"Slidell, LA",30.275195,-89.781174,NaN


# Distances between MSAs

In [186]:
# Driving distance based on averages from Google Maps
dist_pickle = '../data/pickled/distance_df.pickle'
with open(dist_pickle, 'rb') as file:
    dist_df = pickle.load(file)

In [187]:
# Note that I currently kept 2 measures for each city pair
# For example, Chicago->New York and New York->Chicago
# This is because the distance and duration isn't the same, so may want to average them
dist_df.head()

,Origin,Destination,Distance_meters,Distance_miles,Duration_seconds,Duration_minutes
0,"Abilene, TX","Akron, OH",2137508.0,1328.185483,69643.0,1161.0
1,"Abilene, TX","Albany, GA",1582948.0,983.597982,54532.0,909.0
2,"Abilene, TX","Albany, OR",3077199.0,1912.082220,104836.0,1747.0
3,"Abilene, TX","Albany, NY",2917051.0,1812.570897,94703.0,1578.0
4,"Abilene, TX","Albuquerque, NM",783883.0,487.082164,26964.0,449.0


In [188]:
# Create dataframe to look at city pairs
city_pairs = dist_df.copy()
city_pairs['CityPair'] = city_pairs.apply(lambda x: tuple(sorted([x['Origin'], x['Destination']])), axis=1)
city_pairs = city_pairs.drop(columns=['Origin','Destination'])
# Groupby
city_pairs = city_pairs.groupby('CityPair').mean().reset_index()
city_pairs

,CityPair,Distance_meters,Distance_miles,Duration_seconds,Duration_minutes
0,"(Abilene, TX, Akron, OH)",2136773.5,1327.729086,69543.0,1159.0
1,"(Abilene, TX, Albany, GA)",1581900.0,982.946785,54590.0,910.0
2,"(Abilene, TX, Albany, NY)",2916235.0,1812.063858,94709.0,1578.5
3,"(Abilene, TX, Albany, OR)",3078305.5,1912.769767,104797.5,1746.5
4,"(Abilene, TX, Albuquerque, NM)",784616.5,487.537939,26933.5,448.5
...,...,...,...,...,...
74686,"(York, PA, Yuba City, CA)",4333506.0,2692.714957,141599.0,2360.0
74687,"(York, PA, Yuma, AZ)",3927261.5,2440.286406,129393.5,2156.5
74688,"(Youngstown, OH, Yuba City, CA)",3893197.0,2419.119713,126768.5,2112.5
74689,"(Youngstown, OH, Yuma, AZ)",3584812.0,2227.498217,117245.5,1954.0


In [189]:
# Get the "as the crow flies" distance
def get_acf_dist(cp,msa_df):
    c1 = msa_df.loc[msa_df['MainCity']==cp[0], ['lat','lng']]
    c1 = (c1['lat'].values[0],c1['lng'].values[0])
    c2 = msa_df.loc[msa_df['MainCity']==cp[1], ['lat','lng']]
    c2 = (c2['lat'].values[0],c2['lng'].values[0])
    dist = geodesic(c1,c2).miles

    return dist

city_pairs['ACF_Distance_miles'] = city_pairs['CityPair'].apply(lambda cp: get_acf_dist(cp,msa_df))
    

In [190]:
city_pairs.head()

,CityPair,Distance_meters,Distance_miles,Duration_seconds,Duration_minutes,ACF_Distance_miles
0,"(Abilene, TX, Akron, OH)",2136773.5,1327.729086,69543.0,1159.0,1169.268938
1,"(Abilene, TX, Albany, GA)",1581900.0,982.946785,54590.0,910.0,915.647527
2,"(Abilene, TX, Albany, NY)",2916235.0,1812.063858,94709.0,1578.5,1581.642374
3,"(Abilene, TX, Albany, OR)",3078305.5,1912.769767,104797.5,1746.5,1511.136717
4,"(Abilene, TX, Albuquerque, NM)",784616.5,487.537939,26933.5,448.5,437.507433


# Flights between MSAs

In [71]:
# Flight data between MSAs
# Each row represents the sum for a specific flight route for the month.
# Data covers Jan-Apr 2024
flight_pickle = '../data/pickled/flights_df.pickle'
with open(flight_pickle, 'rb') as file:
    flights_df = pickle.load(file)

In [72]:
flights_df.head()

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,REGION,CARRIER_GROUP_NEW,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_CITY_MARKET_ID,AIRCRAFT_GROUP,AIRCRAFT_TYPE,MONTH,YEAR,DISTANCE_GROUP,CLASS,CityPair
215,0.0,1.0,5.0,4.0,233.0,60.0,42.0,3EQ,21974,"Scott Aviation, LLC d/b/a Silver Air",D,2,LAX,"Los Angeles, CA",32575,HSH,"Las Vegas, NV",32211,6,685,4,2024,1,L,"(Las Vegas, NV, Los Angeles, CA)"
539,0.0,1.0,6.0,1.0,247.0,60.0,42.0,3EQ,21974,"Scott Aviation, LLC d/b/a Silver Air",D,2,HOU,"Houston, TX",31453,ADS,"Dallas, TX",30194,0,94,4,2024,1,L,"(Dallas, TX, Houston, TX)"
540,0.0,1.0,6.0,1.0,247.0,60.0,48.0,3EQ,21974,"Scott Aviation, LLC d/b/a Silver Air",D,2,ADS,"Dallas, TX",30194,HOU,"Houston, TX",31453,0,94,4,2024,1,L,"(Dallas, TX, Houston, TX)"
541,0.0,1.0,6.0,1.0,268.0,54.0,48.0,3EQ,21974,"Scott Aviation, LLC d/b/a Silver Air",D,2,RNO,"Reno, NV",34570,PRB,"Paso Robles, CA",34236,6,639,4,2024,1,L,"(Reno, NV, San Luis Obispo, CA)"
548,0.0,1.0,6.0,1.0,510.0,96.0,84.0,3EQ,21974,"Scott Aviation, LLC d/b/a Silver Air",D,2,SAF,"Santa Fe, NM",34674,LAS,"Las Vegas, NV",32211,0,94,4,2024,2,L,"(Las Vegas, NV, Santa Fe, NM)"


In [ ]:
# Note that DISTANCE is already averaged for each trip
# RAMP_TO_RAMP and AIR_TIME are the cumulative for all the trips
# To get average, divide by DEPARTURES_PERFORMED

In [73]:
flights_df[flights_df['CityPair']==('Las Vegas, NV', 'Los Angeles, CA')]

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,REGION,CARRIER_GROUP_NEW,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_CITY_MARKET_ID,AIRCRAFT_GROUP,AIRCRAFT_TYPE,MONTH,YEAR,DISTANCE_GROUP,CLASS,CityPair
215,0.0,1.0,5.0,4.0,233.0,60.0,42.0,3EQ,21974,"Scott Aviation, LLC d/b/a Silver Air",D,2,LAX,"Los Angeles, CA",32575,HSH,"Las Vegas, NV",32211,6,685,4,2024,1,L,"(Las Vegas, NV, Los Angeles, CA)"
1928,0.0,1.0,10.0,1.0,236.0,54.0,42.0,AN,21894,"ADVANCED AIR, LLC",D,6,LAX,"Los Angeles, CA",32575,LAS,"Las Vegas, NV",32211,6,609,2,2024,1,L,"(Las Vegas, NV, Los Angeles, CA)"
1929,0.0,1.0,10.0,1.0,236.0,60.0,48.0,AN,21894,"ADVANCED AIR, LLC",D,6,LAS,"Las Vegas, NV",32211,LAX,"Los Angeles, CA",32575,6,609,2,2024,1,L,"(Las Vegas, NV, Los Angeles, CA)"
8439,0.0,1.0,156.0,123.0,236.0,78.0,42.0,G4,20368,Allegiant Air,D,3,LAS,"Las Vegas, NV",32211,LAX,"Los Angeles, CA",32575,6,698,3,2024,1,L,"(Las Vegas, NV, Los Angeles, CA)"
8638,0.0,1.0,70.0,45.0,236.0,0.0,0.0,AC,19531,Air Canada,I,0,LAX,"Los Angeles, CA",32575,LAS,"Las Vegas, NV",32211,6,694,3,2024,1,L,"(Las Vegas, NV, Los Angeles, CA)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134712,98.0,97.0,13871.0,12443.0,236.0,6667.0,4106.0,WN,19393,Southwest Airlines Co.,D,3,LAS,"Las Vegas, NV",32211,LAX,"Los Angeles, CA",32575,6,612,1,2024,1,F,"(Las Vegas, NV, Los Angeles, CA)"
135798,116.0,112.0,14781.0,13200.0,236.0,8021.0,4829.0,DL,19790,Delta Air Lines Inc.,D,3,LAX,"Los Angeles, CA",32575,LAS,"Las Vegas, NV",32211,6,698,2,2024,1,F,"(Las Vegas, NV, Los Angeles, CA)"
135799,116.0,112.0,14784.0,13459.0,236.0,8489.0,5092.0,DL,19790,Delta Air Lines Inc.,D,3,LAS,"Las Vegas, NV",32211,LAX,"Los Angeles, CA",32575,6,698,2,2024,1,F,"(Las Vegas, NV, Los Angeles, CA)"
135945,118.0,117.0,15441.0,13831.0,236.0,8370.0,5152.0,DL,19790,Delta Air Lines Inc.,D,3,LAX,"Los Angeles, CA",32575,LAS,"Las Vegas, NV",32211,6,698,1,2024,1,F,"(Las Vegas, NV, Los Angeles, CA)"
